### Effective Mandays % - West Clients - Souvik

In [101]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [102]:
import warnings
warnings.filterwarnings("ignore", message=".*SQLAlchemy connectable.*")

In [103]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:\Users\Administrator\LIS\Kenvue\python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [104]:
# Python Master for ISPStore
sheet = client.open_by_key("10tpkruv13OLpXYCgiua0Vnn_QEqWWVq9D9JWpXBNTn4") # Master Sheets ( Dimensions Sheet)

In [105]:
Schema_Master = sheet.worksheet("Schema_Master")
Schema_Master = get_as_dataframe(Schema_Master)
Data = sheet.worksheet("Data")

In [106]:
Schema_Master= Schema_Master[Schema_Master['flag']==1]
Schema_Master.head()

,Project,SQL Schema Name,From Date,To Date,flag,Region
0,Amul GT & MT,[AmulManagement],1-Oct-25,31-Oct-25,1.0,West
1,AWL Agri-GT,[AdaniWilmerManagement],1-Oct-25,31-Oct-25,1.0,West
7,Cholayil,[CholayilManagement],1-Oct-25,31-Oct-25,1.0,West
9,Danone-GT,[DanoneGTManagement],1-Oct-25,31-Oct-25,1.0,West
10,Danone-MT,[DanoneMTManagement],1-Oct-25,31-Oct-25,1.0,West


In [107]:
x= Schema_Master['SQL Schema Name'].to_list()
projects = [str(p).strip("[]") for p in Schema_Master['Project'].tolist()]

In [108]:
date_ranges = [
    (
        pd.to_datetime(row["From Date"]).strftime("%Y-%m-%d"),
        pd.to_datetime(row["To Date"]).strftime("%Y-%m-%d")
    )
    for _, row in Schema_Master.iterrows()
]


In [109]:
server = "157.119.230.120,4071"
database = "PowerBI_LISV2"
username = "sa"
password = "Y0m@Sql.!.123"

conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
TrustServerCertificate=yes;
"""

In [110]:
conn = pyodbc.connect(conn_str)

In [111]:
final_df = pd.DataFrame()

In [112]:
for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
        query = f"""
WITH CTE AS (
    SELECT 
        '{project}' AS SchemaName,
        (TXT.UserName + '-' + TXT.Visit_Date) AS UserVisitKey,
        TXT.Visit_Date,TXT.UserName,
        (TXT.Visit_Date + ' ' + TXT.CHECKIN_TIME)  AS CheckinTime,
		TXT.CHECKIN_TIME,TXT.LISStoreCode,
        (TXT.Visit_Date + '-' + TXT.LISStoreCode + '-' + TXT.UserName) AS StoreDateKey,
        TXT.StoreName,
        TXT.Designation,
        (TXT.Visit_Date + ' ' + TXT.CHECKOUT_TIME) AS CheckoutTime,
        TXT.CHECKOUT_TIME,
		TXT.FullName,
        TXT.NoPermissionReason,
        1 AS VisitFlag,
        DATEPART(DAY, TRY_CONVERT(date, TXT.Visit_Date, 105)) AS DayNum
    FROM (
        SELECT
            FORMAT(CONVERT(datetime, SA.[VisitDate], 105), 'dd-MM-yyyy') AS Visit_Date,
            SA.LISStoreCode,
            SA.StoreName,
            SA.UserName,
            SA.Designation,
            SA.FullName,
            CONVERT(varchar(8), CAST([CheckIn-Time]  AS time), 108) AS CHECKIN_TIME,
            CONVERT(varchar(8), CAST([CheckOut-Time] AS time), 108) AS CHECKOUT_TIME,
            SA.NoPermissionReason
        FROM [PowerBI_LISV2].{value} .[ISPStoreAttendance_NormalExport] AS SA
        LEFT JOIN (
            SELECT K.*
            FROM (SELECT *,
                       RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                FROM PowerBI_LISV2.{value} .UserMaster
            ) AS K
            WHERE K.RNK = 1
        ) AS UM ON UM.UserName = SA.UserName
        WHERE 
            -- current month year filter
            [Month] = MONTH(EOMONTH(GETDATE()))
            AND [Year]  = YEAR(EOMONTH(GETDATE()))
			-- True is considered here for all active users 
            -- AND UM.Status = 'True'
            AND SA.UserName NOT LIKE '%Star%'
            AND LOWER(SA.UserName) NOT LIKE '%test%'
    ) AS TXT
)
SELECT K.*,SUP.Supervisor
FROM
(SELECT cte.SchemaName,cte.Visit_Date,cte.UserName,cte.FullName,cte.Designation,
count(cte.LISStoreCode) as StoreCount,
min(CHECKIN_TIME) as Store_CheckIn,max(CHECKOUT_TIME) as Store_CheckOut 
FROM CTE
-- WHERE cte.Visit_Date = '27-10-2025'
-- WHERE TRY_CONVERT(date, cte.Visit_Date, 105) BETWEEN '{date_from}' AND '{date_to}'
GROUP BY cte.SchemaName,cte.Visit_Date,cte.UserName,cte.FullName,cte.Designation) AS K
LEFT JOIN (SELECT ISP,Supervisor FROM
		(SELECT *,RANK() OVER (PARTITION BY UH.ISP ORDER BY UH.UserHierarchyListId DESC) AS RK FROM
			[PowerBI_LISV2].{value}.UserHierarchyListReport AS UH)AS UH WHERE UH.RK =1) AS SUP
ON SUP.ISP = K.UserName;"""
        df = pd.read_sql(query, conn)
        final_df = pd.concat([final_df, df], ignore_index=True)
        print(f"Running for: {value} {date_from} {date_to}")



Running for: [AmulManagement] 2025-10-01 2025-10-31
Running for: [AdaniWilmerManagement] 2025-10-01 2025-10-31
Running for: [CholayilManagement] 2025-10-01 2025-10-31
Running for: [DanoneGTManagement] 2025-10-01 2025-10-31
Running for: [DanoneMTManagement] 2025-10-01 2025-10-31
Running for: [GCPLManagement] 2025-10-01 2025-10-31
Running for: [GCPLFoodsManagement] 2025-10-01 2025-10-31
Running for: [JohnsonJohnsonManagement] 2025-10-01 2025-10-31
Running for: [ParleManagement] 2025-10-01 2025-10-31
Running for: [PidiliteManagement] 2025-10-01 2025-10-31
Running for: [PiramalManagement] 2025-10-01 2025-10-31
Running for: [ZydusMTManagement] 2025-10-01 2025-10-31
Running for: [ZydusGTManagement] 2025-10-01 2025-10-31


In [113]:
final_df.head()

,SchemaName,Visit_Date,UserName,FullName,Designation,StoreCount,Store_CheckIn,Store_CheckOut,Supervisor
0,Amul GT & MT,01-11-2025,AMLMR027,Tejaswini Sachin Shedage,Merchandiser,3,11:45:50,20:48:32,AML-SUP-11
1,Amul GT & MT,02-11-2025,AMLMR027,Tejaswini Sachin Shedage,Merchandiser,1,11:09:08,20:21:49,AML-SUP-11
2,Amul GT & MT,01-11-2025,AMLMR040,Hamsa,Merchandiser,1,06:28:34,16:16:50,AML-SUP-01
3,Amul GT & MT,02-11-2025,AMLMR040,Hamsa,Merchandiser,1,11:31:22,20:39:25,AML-SUP-01
4,Amul GT & MT,01-11-2025,AMLMR055,Neerja Kashiram Gupta,Merchandiser,2,08:25:31,16:44:13,AML-SUP-11


In [114]:
# # Convert to datetime.time (using a dummy date)
# final_df['Store_CheckIn1'] = pd.to_datetime(final_df['Store_CheckIn'], format='%H:%M:%S')
# final_df['Store_CheckOut1'] = pd.to_datetime(final_df['Store_CheckOut'], format='%H:%M:%S')

# # # Convert timedelta to hours
# # final_df['Work_Hours'] = (final_df['Store_CheckOut'] - final_df['Store_CheckIn']).dt.total_seconds() / 3600
# # # Convert duration to HH:MM format
# final_df['FOS'] = (final_df['Store_CheckOut1'] - final_df['Store_CheckIn1']).apply(lambda x: f"{int(x.total_seconds()//3600):02d}:{int((x.total_seconds()%3600)//60):02d}")

# # Create flag for > 6:30 hours
# final_df['FOS_MoreThan_6_30'] = (final_df['Store_CheckOut1'] - final_df['Store_CheckIn1']).apply(lambda x: 1 if x.total_seconds() > (6.5 * 3600) else 0)

In [115]:

# Convert to datetime (dummy date added automatically)
final_df['Store_CheckIn1'] = pd.to_datetime(final_df['Store_CheckIn'], format='%H:%M:%S', errors='coerce')
final_df['Store_CheckOut1'] = pd.to_datetime(final_df['Store_CheckOut'], format='%H:%M:%S', errors='coerce')

# Calculate duration safely (handle NaT)
def calculate_duration(row):
    if pd.notnull(row['Store_CheckIn1']) and pd.notnull(row['Store_CheckOut1']):
        diff = row['Store_CheckOut1'] - row['Store_CheckIn1']
        hours = int(diff.total_seconds() // 3600)
        minutes = int((diff.total_seconds() % 3600) // 60)
        return f"{hours:02d}:{minutes:02d}"
    else:
        return np.nan

final_df['FOS'] = final_df.apply(calculate_duration, axis=1)

# Create flag for > 6:30 hours safely
def flag_more_than_6_30(row):
    if pd.notnull(row['Store_CheckIn1']) and pd.notnull(row['Store_CheckOut1']):
        diff = row['Store_CheckOut1'] - row['Store_CheckIn1']
        return 1 if diff.total_seconds() > (6.5 * 3600) else 0
    else:
        return np.nan

final_df['FOS_MoreThan_6_30'] = final_df.apply(flag_more_than_6_30, axis=1)


In [116]:
final_df.drop(columns=['Store_CheckIn1', 'Store_CheckOut1'], inplace=True)

In [117]:
final_df.head()

,SchemaName,Visit_Date,UserName,FullName,Designation,StoreCount,Store_CheckIn,Store_CheckOut,Supervisor,FOS,FOS_MoreThan_6_30
0,Amul GT & MT,01-11-2025,AMLMR027,Tejaswini Sachin Shedage,Merchandiser,3,11:45:50,20:48:32,AML-SUP-11,09:02,1
1,Amul GT & MT,02-11-2025,AMLMR027,Tejaswini Sachin Shedage,Merchandiser,1,11:09:08,20:21:49,AML-SUP-11,09:12,1
2,Amul GT & MT,01-11-2025,AMLMR040,Hamsa,Merchandiser,1,06:28:34,16:16:50,AML-SUP-01,09:48,1
3,Amul GT & MT,02-11-2025,AMLMR040,Hamsa,Merchandiser,1,11:31:22,20:39:25,AML-SUP-01,09:08,1
4,Amul GT & MT,01-11-2025,AMLMR055,Neerja Kashiram Gupta,Merchandiser,2,08:25:31,16:44:13,AML-SUP-11,08:18,1


In [118]:
final_df_attendance = pd.DataFrame()

In [119]:
for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
        query = f"""
WITH CTE AS (
    SELECT
        '{project}' AS SchemaName,
        A.UserName,
        M.EmployeeID,
        M.FullName,
        M.Gender,
        M.Region,
        M.State,
        M.City,
        M.Designation,
        TRY_CONVERT(date, M.LastWorkingDate, 105) AS LastWorkingDate,
        M.Status AS EmpStatus,
        -- A.Status AS AttRawStatus,
        -- A.ApprovalStatus,
        CASE
            WHEN A.Status = 'Present' THEN 'P'
            WHEN A.Status = 'Absent' OR A.Status IS NULL THEN 'A'
            WHEN A.Status = 'Leave' THEN 'L'
            WHEN A.Status IN ('Compensatory Off','Comp Off') THEN 'CO'
            WHEN A.Status IN ('Week Off','WeekOff') THEN 'WO'
            WHEN A.Status IN ('Meeting','Meeting/Training') THEN 'M'
            WHEN A.Status = 'Training' THEN 'T'
            WHEN A.Status IN ('National Holiday','National Hoilday','Holiday','Regional Holiday') THEN 'H'
            WHEN A.Status IN ('Bandh/Strike','Bandh Strike','Bandh / Strike','Market closed','Strike') THEN 'B'
            WHEN A.Status = 'Vacant' THEN 'V'
            WHEN A.Status IN ('Personal','Personal Leave') THEN 'PL'
            WHEN A.Status = 'Casual Leave' THEN 'CL'
            WHEN A.Status = 'Half Day' THEN 'HD'
            ELSE 'Condition not found'
        END AS AttStatus, 
        FORMAT(CONVERT(datetime, A.[Date], 105), 'dd-MM-yyyy') AS Date2,
        -- TRY_CONVERT(date, A.[Date], 105) AS Date2,
        RANK() OVER (PARTITION BY A.UserName,A.[Date] ORDER BY A.ISPAttendanceMasterExportId DESC) AS rk
    FROM PowerBI_LISV2.{value}.ISPAttendanceMasterExport AS A
    LEFT JOIN (SELECT K.* FROM
        (SELECT  *, RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK FROM 
        PowerBI_LISV2.{value}.UserMaster) AS K
        WHERE K.RNK =1) AS M
        ON A.UserName = M.UserName
    WHERE 
        -- TRY_CONVERT(date, A.[Date], 105) BETWEEN '{date_from}' AND '{date_to}'
		[Month] = MONTH(EOMONTH(GETDATE())) AND [Year]  = YEAR(EOMONTH(GETDATE()))
        AND M.UserRole NOT IN ('MIS','RSM','SO','Program Manager','Assistant Manager','FO','Office Staff','ASM',
        'Senior Manager','Supervisor','National')
        AND LOWER(A.UserName) NOT LIKE '%test%'
        AND LOWER(A.UserName) NOT LIKE '%sup%'
        AND LOWER(A.FullName) NOT LIKE '%test%')
SELECT * FROM CTE WHERE CTE.rk=1;"""
        df = pd.read_sql(query, conn)
        final_df_attendance = pd.concat([final_df_attendance, df], ignore_index=True)
        print(f"Running for: {value} ")



Running for: [AmulManagement] 
Running for: [AdaniWilmerManagement] 
Running for: [CholayilManagement] 
Running for: [DanoneGTManagement] 
Running for: [DanoneMTManagement] 
Running for: [GCPLManagement] 
Running for: [GCPLFoodsManagement] 
Running for: [JohnsonJohnsonManagement] 
Running for: [ParleManagement] 
Running for: [PidiliteManagement] 
Running for: [PiramalManagement] 
Running for: [ZydusMTManagement] 
Running for: [ZydusGTManagement] 


In [120]:
final_df_attendance.head()

,SchemaName,UserName,EmployeeID,FullName,Gender,Region,State,City,Designation,LastWorkingDate,EmpStatus,AttStatus,Date2,rk
0,Amul GT & MT,AMLMR027,YM2023000754,Tejaswini Sachin Shedage,F,West,Maharashtra,Mumbai,Merchandiser,None,True,P,01-11-2025,1
1,Amul GT & MT,AMLMR027,YM2023000754,Tejaswini Sachin Shedage,F,West,Maharashtra,Mumbai,Merchandiser,None,True,P,02-11-2025,1
2,Amul GT & MT,AMLMR029,YM2023000856,Sachin Subhash Shukla,M,West,Maharashtra,Mumbai,Merchandiser,None,True,L,01-11-2025,1
3,Amul GT & MT,AMLMR029,YM2023000856,Sachin Subhash Shukla,M,West,Maharashtra,Mumbai,Merchandiser,None,True,L,02-11-2025,1
4,Amul GT & MT,AMLMR040,YM2023002962,Hamsa,M,South,Karnataka,Bangalore,Merchandiser,None,True,P,01-11-2025,1


In [121]:
final_emd_df = pd.merge(final_df_attendance[['SchemaName','UserName','EmpStatus','Date2','AttStatus']],final_df,left_on=['SchemaName','UserName','Date2'], right_on=['SchemaName','UserName','Visit_Date'],how = 'left')

In [122]:
final_emd_df=final_emd_df.fillna('')

In [123]:
final_emd_df.head()

,SchemaName,UserName,EmpStatus,Date2,AttStatus,Visit_Date,FullName,Designation,StoreCount,Store_CheckIn,Store_CheckOut,Supervisor,FOS,FOS_MoreThan_6_30
0,Amul GT & MT,AMLMR027,True,01-11-2025,P,01-11-2025,Tejaswini Sachin Shedage,Merchandiser,3.0,11:45:50,20:48:32,AML-SUP-11,09:02,1.0
1,Amul GT & MT,AMLMR027,True,02-11-2025,P,02-11-2025,Tejaswini Sachin Shedage,Merchandiser,1.0,11:09:08,20:21:49,AML-SUP-11,09:12,1.0
2,Amul GT & MT,AMLMR029,True,01-11-2025,L,,,,,,,,,
3,Amul GT & MT,AMLMR029,True,02-11-2025,L,,,,,,,,,
4,Amul GT & MT,AMLMR040,True,01-11-2025,P,01-11-2025,Hamsa,Merchandiser,1.0,06:28:34,16:16:50,AML-SUP-01,09:48,1.0


In [124]:
final_emd_df.to_csv('souvikfinal_emd_df.csv')

In [125]:
#Sending df on sheet
Data.batch_clear(["A1"])
set_with_dataframe(Data, final_emd_df)